In [1]:
# coding: utf-8
__author__ = 'Sandro Vega Pons : https://www.kaggle.com/svpons'

'''Partially based on grid_plus_classifier script:
https://www.kaggle.com/svpons/facebook-v-predicting-check-ins/grid-plus-classifier
'''

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier


def prepare_data(df, n_cell_x, n_cell_y):
    """
    Feature engineering and computation of the grid.
    """
    #Creating the grid
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    eps = 0.00001  
    xs = np.where(df.x.values < eps, 0, df.x.values - eps)
    ys = np.where(df.y.values < eps, 0, df.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df['grid_cell'] = (pos_y * n_cell_x + pos_x).astype(np.int16)
    
    #Feature engineering
    fw = [500, 1000, 4, 3, 1./22., 2, 10] #feature weights (black magic here)
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour * fw[2]).astype(np.int8)
    df['weekday'] = (d_times.weekday * fw[3]).astype(np.int8)
    df['day'] = (d_times.dayofyear * fw[4]).astype(np.int8)
    df['month'] = (d_times.month * fw[5]).astype(np.int8)
    df['year'] = ((d_times.year - 2013) * fw[6]).astype(np.int8)

    df = df.drop(['time'], axis=1) 
    return df
    

def process_one_cell(df_train, df_test, grid_id, th):
    """   
    Classification inside one grid cell.
    """   
    #Working on df_train
    df_cell_train = df_train.loc[df_train.grid_cell == grid_id]
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]

    #Working on df_test
    df_cell_test = df_test.loc[df_test.grid_cell == grid_id]
    row_ids = df_cell_test.index
    
    #Preparing data
    le = LabelEncoder()
    y = le.fit_transform(df_cell_train.place_id.values)
    X = df_cell_train.drop(['place_id', 'grid_cell'], axis=1).values.astype(int)
    X_test = df_cell_test.drop(['grid_cell'], axis = 1).values.astype(int)
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=25, weights='distance', 
                               metric='manhattan')
    clf.fit(X, y)
    y_pred = clf.predict_proba(X_test)
    #pred_probas = np.argsort(y_pred, axis=1)[:,::-1][:,:3]
    pred_labels = le.inverse_transform(np.argsort(y_pred, axis=1)[:,::-1][:,:3])
    
    return pred_labels, row_ids, y_pred
    
    
def process_grid(df_train, df_test, th, n_cells):
    """
    Iterates over all grid cells, aggregates the results and makes the
    submission.
    """ 
    preds = np.zeros((df_test.shape[0], 3), dtype=int)
    
    for g_id in range(1):
        if g_id % 100 == 0:
            print('iter: %s' %(g_id))
        
        #Applying classifier to one grid cell
        pred_labels, row_ids, y_pred = process_one_cell(df_train, df_test, g_id, th)

        #Updating predictions
        preds[row_ids] = pred_labels

    print('Generating submission file ...')
    #Auxiliary dataframe with the 3 best predictions for each sample
    df_aux = pd.DataFrame(preds, dtype=str, columns=['l1', 'l2', 'l3'])  
    
    #Concatenating the 3 predictions for each sample
    ds_sub = df_aux.l1.str.cat([df_aux.l2, df_aux.l3], sep=' ')
    
    #Writting to csv
    ds_sub.name = 'place_id'
    ds_sub.to_csv('sub_grid.csv', index=True, header=True, index_label='row_id') 
    
    return y_pred, pred_labels,preds
      

In [9]:
"""
"""
print('Loading data ...')
dfgrid = pd.read_hdf('grid0predictedplacesreplaced__grid.h5','table')
dfall = pd.read_hdf('grid0predictedplacesreplaced__all.h5','table')
dforig = pd.read_hdf('grid0predictedplaces_.h5','table')
df_test = pd.read_hdf('test.h5','table')

Loading data ...


In [11]:
predsorig = np.zeros((df_test.shape[0], 3), dtype=int)
predsgrid = np.zeros((df_test.shape[0], 3), dtype=int)
predsall = np.zeros((df_test.shape[0], 3), dtype=int)

In [17]:
idd = dforig.index

In [18]:
idd

Int64Index([      5,     442,    2133,    2794,    3200,    4345,    4532,
               4758,    5443,    5991,
            ...
            8601563, 8601827, 8602102, 8602382, 8603444, 8604653, 8605309,
            8605991, 8606808, 8606934],
           dtype='int64', length=11290)

In [19]:
predsorig[idd] = dforig.values
predsgrid[idd] = dfgrid.values
predsall[idd] = dfall.values

In [22]:
df_aux1 = pd.DataFrame(predsorig, dtype=str, columns=['l1', 'l2', 'l3'])  
df_aux2 = pd.DataFrame(predsgrid, dtype=str, columns=['l1', 'l2', 'l3'])
df_aux3 = pd.DataFrame(predsall, dtype=str, columns=['l1', 'l2', 'l3'])
#Concatenating the 3 predictions for each sample
ds_sub1 = df_aux1.l1.str.cat([df_aux1.l2, df_aux1.l3], sep=' ')
ds_sub2 = df_aux2.l1.str.cat([df_aux2.l2, df_aux2.l3], sep=' ')
ds_sub3 = df_aux3.l1.str.cat([df_aux3.l2, df_aux3.l3], sep=' ')

#Writting to csv
ds_sub1.name = 'place_id'
ds_sub1.to_csv('grid0predictedplaces_knn.csv', index=True, header=True, index_label='row_id')

ds_sub2.name = 'place_id'
ds_sub2.to_csv('grid0predictedplacesreplaced__grid_knn.csv', index=True, header=True, index_label='row_id')

ds_sub3.name = 'place_id'
ds_sub3.to_csv('grid0predictedplacesreplaced__all_knn.csv', index=True, header=True, index_label='row_id')